In [1]:
from sklearn.model_selection import KFold
import lightgbm

import multiprocessing 
from multiprocessing import Pool 
from functools import partial 
from data_loader_v2 import data_loader_v2
import os 
import pandas as pd
import numpy as np
import joblib

train_folder = '../데이콘 스터디/train/'
test_folder = '../데이콘 스터디/test/'
train_label_path = '../데이콘 스터디/train_label.csv'

train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

In [2]:
# 모든 csv 파일의 상태_B로 변화는 시점이 같다라고 가정
# 하지만, 개별 csv파일의 상태_B로 변화는 시점은 상이할 수 있음


def data_loader_all_v2(func, files, folder='', train_label=None, event_time=10, nrows=60):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)  
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close() 
        pool.join() 
    combined_df = pd.concat(df_list)    
    return combined_df


In [3]:
import time
start_time = time.time()


train = data_loader_all_v2(data_loader_v2, train_list, folder=train_folder, train_label=train_label, event_time=10, nrows=60)
test = data_loader_all_v2(data_loader_v2, test_list, folder=test_folder, train_label=None, event_time=20, nrows=60)

y = train['label']
train.drop('label',axis=1,inplace=True)

print("--- %s, sconds ---" % (time.time()-start_time))

--- 268.70541167259216, sconds ---


In [5]:
train.loc[:, train.nunique() != 1].shape[1]

3326

In [32]:
train_2=train.loc[:, train.nunique() == 1]
a=[]
for i in range(train_2.shape[1]):
 a.append(train_2.iloc[:,i].std())


In [31]:
# nunique !=1 함수로 뽑은 컬럼의 값이 0이 아닌값들이 존재한다.
a.sort()
a[-10:]

[6.208797501674418e-11,
 1.310256648327721e-10,
 1.53734885266912e-10,
 3.981361199517292e-10,
 3.981361199517292e-10,
 3.981361199517292e-10,
 3.981361199517292e-10,
 8.112790839450428e-10,
 8.112790839450428e-10,
 2.2357832907141948e-07]

In [39]:
## 값이 한개인 값을 제외한 컬럼을 뽑으면 3326개 < 편차가 0인 값을 제외한 컬럼을 뽑으면 3416
## 아마 값이 미묘하게 다른데도 그 차이가 너무 작아서 nunique가 1로 나오는 듯

print('nunique !=1 함수를 이용한 컬럼 개수 : ',train.loc[1,train.nunique()!=1].shape[1],
      '\n std !=0 함수를 이용한 컬럼 개수 : ',train.loc[1,train.std()!=0].shape[1])

nunique !=1 함수를 이용한 컬럼 개수 :  3326 
 std !=0 함수를 이용한 컬럼 개수 :  3416


# nunique 함수로 쓸모 없는 컬럼을 제거하여 모델링 했을 때, 정확도가 떨어지는 문제점의 이유는 ?
## **아주 미세하게 작은 값을 0이라고 인식하지 않는 nunique함수의 문제였음!**
## **따라서 std=0 함수로 편차가 없는 컬럼을 전처리 했더니 베이직 랜덤 포레스트 모델의 정확도가 올라갔음**